In [46]:
from collections import Counter
from gensim.parsing.preprocessing import STOPWORDS
STOPWORDS = list(set(STOPWORDS))
hand_picked_stop_words = ['rt' , "it's" , 'says' , "doesn't" , "shes" ,"hes" , "she's" ,"he's", u"don't" , "thanks" , "thank's", "like" ,
                         "today" , "time", "know" , "knows" , "help" , "check" , "good", 'must', 'back', 'service' ,
                         'trust', 'yesterday' , 'before' , 'away', 'products', "we're", "bad", "its" ,"it's" , "like" ,
                         'tell', 'talk' , 'wait', 'think','thinks','00pm',"jr's", 'truth','want','wants', 'give','gave',
                         'sure', 'edit', 'may', 'maybe', 'may not', 'might','might not', "we've", 'able', 'go', 'goes',
                         'went', "what's", 'list', 'lists', "can't", 'forever', 'ever', 'says', 'item', "we'd", '#deporthillary',
                         'woud', 'will', 'would', 'mmmmk', "t'was", "ira's", 'sehe', 'haaa', "l'art", 'spss', "bryan's"]
STOPWORDS = STOPWORDS + hand_picked_stop_words
import re
import numpy as np
import graphlab as gl
from utils.vector_utils import find_norm , find_similar , vector_from_document_tfidf_dict , vector_from_document_tfidf_mongo
from utils.preprocess import pre_process_sentence , pre_process_tokens


In [5]:
from twitter_preprocess import tweet_parser , deduplication_tweets , preprocess_tweets_test

In [23]:
topic_model = gl.load_model('../Data/topic_model_v_1.0_gl/')

In [129]:
def get_me_the_topic(tweet):
    
    if isinstance(tweet, list):
        tweets_deduplicated = deduplication_tweets(tweet)
        tweet_parsed = tweet_parser(tweets_deduplicated)
        tweet_list = [t.split() for t in tweet_parsed]
        tweet_preprocess_list = map(preprocess_tweets_test , tweet_list)
        ######### Remove any null tweets
        final_tweets = []
        tweet_index = []
        for i,t in enumerate(tweet_preprocess_list):
            if t:
                final_tweets.append(t)
                tweet_index.append(i)
        sf_tweet = gl.SFrame(final_tweets)
        tweet_docs = gl.text_analytics.count_words(sf_tweet['X1'])
        return tweets_deduplicated , tweet_docs , tweet_index , final_tweets
    
            
    

In [174]:
import os
import glob
import json
path = '../Data/twitter_categ_data/'
data_holder_dict = {}
for infile in glob.glob( os.path.join(path, '*.json') ):
    data_holder_dict[infile.split('/')[-1].split('.')[0]] = json.load(open(infile))
    print "current file is: " + infile
    

In [175]:
data_holder_dict.keys()

['Health & Fitness',
 'Business',
 'Government',
 'Telecom',
 'automobile',
 'Travel',
 'Sports',
 'Food & Drink',
 'Personal Finance',
 'Arts&Entertainment',
 'Real Estate',
 'technology & computing',
 'Shopping',
 'Classifieds',
 'Style&Fashion',
 'Charity']

In [8]:
tweet_holder = []
for key_ in data_holder_dict.iterkeys():
    sub_key_ = data_holder_dict[key_].keys()[0]
    tweet_data = data_holder_dict[key_][sub_key_]
    for tweet_dict in tweet_data:
        tweet_holder.append(tweet_dict[u'twitter_text'])

In [196]:
tweet_holder = []

tweet_data = data_holder_dict['automobile']['automobile']
for tweet_dict in tweet_data:
    tweet_holder.append(tweet_dict[u'twitter_text'])

In [198]:
TD , tweet_holder_processed , TI , tweet_preprocess = get_me_the_topic(tweet_holder)

In [199]:
predictions = topic_model.predict(tweet_holder_processed)
predictions

dtype: int
Rows: 1683
[3, 14, 18, 3, 18, 3, 19, 7, 14, 19, 14, 19, 19, 14, 3, 17, 3, 20, 18, 3, 18, 18, 3, 16, 14, 3, 0, 17, 14, 14, 15, 14, 14, 3, 14, 20, 3, 15, 11, 4, 14, 18, 20, 19, 3, 14, 14, 19, 3, 3, 3, 19, 7, 14, 20, 20, 18, 3, 19, 17, 14, 1, 19, 19, 20, 3, 15, 14, 3, 20, 14, 19, 3, 14, 3, 3, 14, 20, 20, 20, 16, 13, 15, 17, 17, 19, 16, 10, 14, 12, 18, 14, 14, 2, 15, 14, 2, 3, 3, 3, ... ]

In [166]:
import numpy as np
import pymongo

from utils.vector_utils import find_norm , find_similar , vector_from_document_tfidf_dict , vector_from_document_tfidf_mongo
from utils.preprocess import pre_process_sentence , pre_process_tokens

from sklearn.cluster import MiniBatchKMeans
from collections import defaultdict


glove_twitter_vec = np.load('../Data/glove_twiiter_300_vec.npy')
glove_twitter_vocab = np.load('../Data/glove_vocab.npy')

glove_twitter_dict = dict(zip(glove_twitter_vocab, glove_twitter_vec))

client = pymongo.MongoClient()
DB_NAME = 'AIQX_MONGO'
db = client[DB_NAME]

collection_categ_vec = db.get_collection('category_to_vectors')
collection = collection_categ_vec.find()
all_categ = []
for i in collection:
    all_categ.append(i[u'category'])
    
to_remove = [u'Misc. Other',u'Non Profit',u'Tier 2 Services',u'Non Profit', u'Food and Drink', u'Misleading Claims', u'Get Rich Quick']
new_categ = [i for i in all_categ if i not in to_remove]

pp_dict = {}
for categ in new_categ:
    pp_dict[categ] = pre_process_tokens(categ).split()
word_list = pp_dict.values()
word_list = [w for word in word_list for w in word]
word_list = list(set(word_list))
model_not_know = []
for w in word_list:
    if w not in glove_twitter_dict:
        model_not_know.append(w)
print "Model dont know these words {}".format(model_not_know)

#This is also interesting to try with Ward Hierarchical Clustering

tfidf_dict = {k:1.0 for k in word_list}
categ_vec_dict = {}
for categ, tokens in pp_dict.iteritems():
    categ_vec_dict[categ] = vector_from_document_tfidf_dict(tfidf_dict , tokens , glove_twitter_dict)

X = categ_vec_dict.values()
Y = categ_vec_dict.keys()

clusters = MiniBatchKMeans(n_clusters=35, max_iter=1100,batch_size=100,
                        n_init=1,init_size=2000)

clusters.fit(X)

cluster_dict=defaultdict(list)
for word,label in zip(Y,clusters.labels_):
    cluster_dict[label].append(word)
import cPickle
cPickle.dump(cluster_dict , open('category_clusters.pkl', 'wb') , protocol = 2)



In [223]:
df_wcp_norm = cPickle.load(open('wcp_topic_frame.pkl'))

['mpoints',
 'donate',
 '#mplaces',
 'checked',
 'andy',
 'home',
 'bofa',
 'welcome',
 'love',
 'yass',
 'comms',
 'like',
 'discrimination',
 'activists',
 'muslim',
 'tell',
 'talking',
 'american',
 "today's",
 'winning']

In [168]:
CATEG_MAT = np.array(categ_vec_dict.values())
CATEG_KEY = categ_vec_dict.keys()

In [49]:
a , b = find_similar(find_norm(CATEG_MAT) , find_norm(glove_twitter_dict['today']))

In [ ]:
list(df_wcp_norm['Topic_0'].sort_values(ascending=False)[:20].index)

In [212]:
topic_vec = {}
for i in xrange(topic_model.num_topics):
    words_tweets = list(topic_model.get_topics([i] , 20)['word'])[::-1]
    words_tweets = [w.strip('#') for w in words_tweets ]
    word_tfidf = {w:i+1 for i, w in enumerate(words_tweets) if w in glove_twitter_dict}
    topic_vec[i] = vector_from_document_tfidf_dict( word_tfidf , word_tfidf.keys() , glove_twitter_dict)

In [253]:
topic_vec_wcp = {}
for idx ,col_ in enumerate(df_wcp_norm.columns):
    words_tweets = list(df_wcp_norm[col_].sort_values(ascending=False)[:20].index[::-1])
    words_tweets = [w.strip('#') for w in words_tweets ]
    word_tfidf = {w:i+1 for i, w in enumerate(words_tweets) if w in glove_twitter_dict}
    topic_vec_wcp[idx] = vector_from_document_tfidf_dict( word_tfidf , word_tfidf.keys() , glove_twitter_dict)

In [255]:

write_file = open('topic_report_wcp.txt', 'wb')

for top_ in xrange(topic_model.num_topics):
    a , b = find_similar(find_norm(CATEG_MAT) , find_norm(topic_vec_wcp[top_]))
    topa = a[:3]
    categ = [CATEG_KEY[i] for i in b[:3]]
    res = zip(categ,topa)
    topwords = list(topic_model.get_topics([top_], 20)['word'])
    
    write_file.write('Topic_{}'.format(top_))
    write_file.write('\n')
    write_file.write('Top_words')
    write_file.write('\n')
    write_file.write('___________')
    write_file.write('\n')
    write_file.write(str(topwords))
    write_file.write('\n')
    write_file.write('Top Categories')
    write_file.write('\n')
    write_file.write('___________')
    write_file.write('\n')
    write_file.write(str(res))
    write_file.write('\n')
    write_file.write('****************** \n')
    write_file.write('\n')
 
write_file.close()

In [258]:

write_file = open('topic_report_wcp.txt', 'wb')

for idx ,col_ in enumerate(df_wcp_norm.columns):
    a , b = find_similar(find_norm(CATEG_MAT) , find_norm(topic_vec_wcp[idx]))
    topa = a[:3]
    categ = [CATEG_KEY[i] for i in b[:3]]
    res = zip(categ,topa)
    topwords = list(df_wcp_norm[col_].sort_values(ascending=False)[:20].index)
    
    write_file.write('Topic_{}'.format(idx))
    write_file.write('\n')
    write_file.write('Top_words')
    write_file.write('\n')
    write_file.write('___________')
    write_file.write('\n')
    write_file.write(str(topwords))
    write_file.write('\n')
    write_file.write('Top Categories')
    write_file.write('\n')
    write_file.write('___________')
    write_file.write('\n')
    write_file.write(str(res))
    write_file.write('\n')
    write_file.write('****************** \n')
    write_file.write('\n')
 
write_file.close()

In [200]:
from collections import Counter
tweet_extended = [t for tweet in tweet_preprocess for t in tweet ]

In [201]:
counter = Counter(tweet_extended)

In [202]:
w = sorted ( counter.items() , key = lambda x: x[1] , reverse=True)[:20]

In [203]:

words_tweets = [a for a,b in w]
words_tweets = [w.strip('#') for w in words_tweets ]
word_tfidf = {w:1.0 for i, w in enumerate(words_tweets) if w in glove_twitter_dict}
trend_vec = vector_from_document_tfidf_dict( word_tfidf , word_tfidf.keys() , glove_twitter_dict)

In [204]:
a , b = find_similar(find_norm(CATEG_MAT) , find_norm(trend_vec))
topa = a[:10]
categ = [CATEG_KEY[i] for i in b[:10]]
res = zip(categ,topa)

In [205]:
res

[(u'Sports and Events', 0.83642864),
 (u'Home and Decor', 0.72825092),
 (u'Fitness and Health', 0.72267926),
 (u'Video Games', 0.72193456),
 (u'Sports', 0.7170397),
 (u'Family and Parenting', 0.71623802),
 (u'Online Games', 0.67085707),
 (u'Home&Garden', 0.66232729),
 (u'Tobacco and Smoking Products', 0.64986849),
 (u'Toys/Games', 0.64572543)]

In [206]:
res = {}
for w in word_tfidf:
    a , b = find_similar(find_norm(CATEG_MAT) , find_norm(glove_twitter_dict[w]))
    res[w] = (CATEG_KEY[b[0]], a[0])

In [207]:
res

{u'bolt': (u'Sports and Events', 0.35478362),
 u'canada': (u'Travel Services', 0.55970305),
 u'colorado': (u'Real Estate', 0.46520364),
 u'football': (u'Sports', 0.77181947),
 u'ford': (u'Automotive and Vehicles', 0.55983382),
 u'game': (u'Video Games', 0.79502982),
 u'gold': (u'Jewelry', 0.52809495),
 u'like': (u'Home and Decor', 0.72784287),
 u'mercedes': (u'Automotive and Vehicles', 0.43572327),
 u'paralympics': (u'Sports and Events', 0.37950128),
 u'penny': (u'Deals and Coupons', 0.38812646),
 u'rugby': (u'Sports', 0.63019615),
 u'season': (u'Sports and Events', 0.62155765),
 u'spark': (u'Automotive and Vehicles', 0.41202879),
 u'swimming': (u'Sports and Events', 0.50748211),
 u'team': (u'Sports and Events', 0.61411911),
 u'time': (u'Home and Decor', 0.67333037),
 u'today': (u'Sports and Events', 0.76472807),
 u'usain': (u'Sports and Events', 0.28603327)}

In [171]:
a , b = find_similar(find_norm(glove_twitter_vec) , find_norm(glove_twitter_dict['corbyn']))

In [172]:
[glove_twitter_vocab[i] for i in b[:10]]

['corbyn',
 'caelan',
 'shada',
 'moosey',
 'brigette',
 'kersten',
 'zharfan',
 'ruairi',
 'daylen',
 '\xd8\xad\xd9\x81\xd9\x84\xd9\x86\xd8\xa7']

In [158]:
np.dot(find_norm(glove_twitter_dict['train']) , find_norm(glove_twitter_dict['events']))

0.28977734

In [213]:
#################### Topic Vec ---> Vec Relationship

topic_vec_keys = topic_vec.keys()

In [215]:
topic_vec_values = np.array(topic_vec.values())

In [259]:
a , b = find_similar(find_norm(np.array(topic_vec_wcp.values())) , find_norm(topic_vec_wcp[0]))

In [260]:
a

array([ 1.        ,  0.80880982,  0.77605438,  0.74838299,  0.7306779 ,
        0.72602791,  0.72203177,  0.70262933,  0.68319756,  0.67462134,
        0.67155147,  0.65937942,  0.65381235,  0.64274293,  0.64195365,
        0.63957697,  0.62918758,  0.62296802,  0.59599638,  0.55857086,
        0.55492938], dtype=float32)

In [261]:
b

array([ 0, 15, 19, 18,  3,  1, 17, 16,  8, 14,  5,  7, 11, 20, 12,  9, 13,
        2,  6,  4, 10])

In [264]:
df_wcp_norm['Topic_15'].sort_values(ascending=False)[:10]

digital       0.000025
#jobs         0.000025
click         0.000025
#job          0.000025
latest        0.000025
tech          0.000025
cloud         0.000025
#careerarc    0.000025
#delljobs     0.000025
we're         0.000025
Name: Topic_15, dtype: float64

In [265]:
df_wcp_norm['Topic_0'].sort_values(ascending=False)

#ebay            0.000025
#maga            0.000025
engine           0.000025
#reviews         0.000025
#ideas           0.000025
steel            0.000025
#trump           0.000025
#neverhillary    0.000025
#trumptrain      0.000025
#events          0.000025
#trump2016       0.000025
#apps            0.000025
#hillary         0.000025
#news            0.000025
#america         0.000025
brownlee         0.000025
hamilton         0.000025
#ohio            0.000025
surf             0.000025
#ebayus          0.000025
#dncleak         0.000025
#giveaway        0.000025
#foxnews         0.000025
#products        0.000025
pairs            0.000025
pump             0.000025
motorcycle       0.000025
#deplorables     0.000025
#clinton         0.000025
stereo           0.000025
                   ...   
airport          0.000009
help             0.000009
love             0.000009
hunger           0.000009
want             0.000009
#smallbiz        0.000009
core             0.000009
free        